# J1939 Digital Annex to JSON

In [1]:
import json
import pandas

In [2]:
!pip install --upgrade --user numexpr

Requirement already up-to-date: numexpr in c:\users\jdaily\appdata\roaming\python\python38\site-packages (2.8.4)


In [3]:
!pip install  --upgrade --user openpyxl

Requirement already up-to-date: openpyxl in c:\users\jdaily\appdata\roaming\python\python38\site-packages (3.1.2)


## Load the SLOTs

In [4]:
with open('J1939DA OCT2021.xlsx', 'rb') as io:
    slots_df = pandas.read_excel(io, sheet_name='SLOTs',
                                 usecols="B:P", 
                                 engine='openpyxl',
                                 index_col=0,
                                 header = 3)
slots_df

,SLOT Name,SLOT Type,Scaling,Range,Offset,Length,Unit,Transfer Function\nType,Scale Factor\n(value only),Offset\n(value only),Range Maximum\n(value only),Length Minimum\n(bits),Length Maximum\n(bits),Date Created or Last Modified
SLOT Identifier,,,,,,,,,,,,,,
202,Diagnostic,Diagnostic only,NaN,NaN,NaN,NaN,NaN,diagnostic_only,NaN,NaN,,NaN,NaN,NaT
414,SAEaa01,Angular Acceleration,1 rpm/s per bit,0 to 64 255 rpm/s,0 rpm/s,2 bytes,rpm/s,numeric,1.000000,0.000,64255,16.0,16.0,2015-02-19
115,SAEac01(obs),Acceleration,0.000 488 281 25 m/s² per bit,-15.687 to 15.687 511 718 75 m/s²,-15.687 m/s²,2 bytes,m/s²,numeric,0.000488,-15.687,15.687512,16.0,16.0,2020-11-12
140,SAEac02,Acceleration,0.1 m/s² per bit,-12.5 to 12.5 m/s²,-12.5 m/s²,1 byte,m/s²,numeric,0.100000,-12.500,12.5,8.0,8.0,2000-05-11
303,SAEac03,Acceleration,0.01 m/s² per bit,-320 to 322.55 m/s²,-320 m/s²,2 bytes,m/s²,numeric,0.010000,-320.000,322.55,16.0,16.0,2008-08-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,SAEvr04,"Velocity, rotational",10 rpm per bit,0 to 2 500 rpm,0 rpm,1 byte,rpm,numeric,10.000000,0.000,2500,8.0,8.0,2014-04-30
97,SAEvr05,"Velocity, rotational",32 rpm per bit,0 to 8 000 rpm,0 rpm,1 byte,rpm,numeric,32.000000,0.000,8000,8.0,8.0,2014-04-30
223,SAEvr06,"Velocity, rotational",1 rpm per bit,0 to 64 255 rpm,0 rpm,2 bytes,rpm,numeric,1.000000,0.000,64255,16.0,16.0,2014-04-30


In [5]:
reduced_slots = slots_df[["Length",
                          "Unit",
                          "Scale Factor\n(value only)",
                          "Offset\n(value only)",
                          "Range Maximum\n(value only)",
                          "Length Minimum\n(bits)",
                          "Length Maximum\n(bits)"]].copy()
reduced_slots.rename(columns={"Scale Factor\n(value only)": 'Resolution', 
                             "Offset\n(value only)": 'Offset',
                             "Range Maximum\n(value only)":'OperationalHigh',
                             'Length Minimum\n(bits)':"Length",
                             "Length":"byte length",
                             "Length Maximum\n(bits)":"Max Bits"},
                    inplace=True)

reduced_slots

,byte length,Unit,Resolution,Offset,OperationalHigh,Length,Max Bits
SLOT Identifier,,,,,,,
202,NaN,NaN,NaN,NaN,,NaN,NaN
414,2 bytes,rpm/s,1.000000,0.000,64255,16.0,16.0
115,2 bytes,m/s²,0.000488,-15.687,15.687512,16.0,16.0
140,1 byte,m/s²,0.100000,-12.500,12.5,8.0,8.0
303,2 bytes,m/s²,0.010000,-320.000,322.55,16.0,16.0
...,...,...,...,...,...,...,...
79,1 byte,rpm,10.000000,0.000,2500,8.0,8.0
97,1 byte,rpm,32.000000,0.000,8000,8.0,8.0
223,2 bytes,rpm,1.000000,0.000,64255,16.0,16.0


In [6]:
reduced_slots.groupby('Length').groups

{0.0: [109, 271, 291, 381, 108, 111, 258, 489, 228, 283, 490], 1.0: [86], 2.0: [87], 3.0: [88], 4.0: [89, 220, 395, 268], 5.0: [292, 90, 232, 215], 6.0: [91, 141, 284, 263, 477], 7.0: [92, 260, 216], 8.0: [140, 11, 244, 248, 94, 275, 93, 239, 120, 60, 62, 255, 129, 133, 230, 65, 66, 139, 458, 287, 95, 15, 234, 407, 420, 439, 40, 41, 410, 409, 423, 427, 425, 426, 135, 121, 24, 101, 25, 204, 26, 297, 286, 44, 128, 45, 46, 254, 299, 308, 404, 296, 48, 54, 53, 47, 1, 2, 49, 161, 229, 246, 411, 428, 31, 56, 447, 33, 385, 35, 259, 298, 326, 83, 84, 274, 59, 63, 99, 61, 4, 3, 241, 247, 322, 488, 67, 137, 74, 73, 405, 406, 225, 79, 97], 10.0: [205], 11.0: [218], 12.0: [281, 233, 449, 457, 454, 455], 16.0: [414, 115, 303, 10, 113, 8, 9, 136, 242, 365, 379, 446, 485, 131, 276, 211, 342, 351, 12, 208, 306, 307, 302, 460, 359, 13, 14, 39, 37, 231, 235, 270, 445, 463, 301, 104, 146, 250, 143, 352, 403, 436, 438, 441, 444, 451, 440, 483, 323, 17, 16, 354, 366, 462, 80, 145, 221, 249, 347, 390, 453, 

In [7]:
slot_dict = reduced_slots.to_dict('index')
slot_dict

{202: {'byte length': nan,
  'Unit': nan,
  'Resolution': nan,
  'Offset': nan,
  'OperationalHigh': '',
  'Length': nan,
  'Max Bits': nan},
 414: {'byte length': '2 bytes',
  'Unit': 'rpm/s',
  'Resolution': 1.0,
  'Offset': 0.0,
  'OperationalHigh': 64255,
  'Length': 16.0,
  'Max Bits': 16.0},
 115: {'byte length': '2 bytes',
  'Unit': 'm/s²',
  'Resolution': 0.00048828125,
  'Offset': -15.687,
  'OperationalHigh': 15.68751171875,
  'Length': 16.0,
  'Max Bits': 16.0},
 140: {'byte length': '1 byte',
  'Unit': 'm/s²',
  'Resolution': 0.1,
  'Offset': -12.5,
  'OperationalHigh': 12.5,
  'Length': 8.0,
  'Max Bits': 8.0},
 303: {'byte length': '2 bytes',
  'Unit': 'm/s²',
  'Resolution': 0.01,
  'Offset': -320.0,
  'OperationalHigh': 322.55,
  'Length': 16.0,
  'Max Bits': 16.0},
 7: {'byte length': '4 bytes',
  'Unit': 'deg',
  'Resolution': 1e-07,
  'Offset': -210.0,
  'OperationalHigh': 211.1081215,
  'Length': 32.0,
  'Max Bits': 32.0},
 10: {'byte length': '2 bytes',
  'Unit': '

In [8]:
#Create a pandas data frame with the SPN as the index column and all the data from the SPs and PGs table.
with open('J1939DA OCT2021.xlsx', 'rb') as io:
    spn_df = pandas.read_excel(io, sheet_name='SPs & PGs',
                                 usecols="E:AP", 
                                 header = 3,
                                 converters={'SPN':int,
                                             'PGN':int,
                                             'EDP':int,
                                             'DP':int,
                                             'PF':int,
                                             'SLOT Identifier':int,
                                             'Length Minimum\n(bits)':int,
                                             'Length Maximum\n(bits)':int})
spn_df

C:\Users\jdaily\AppData\Roaming\Python\Python38\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


,PGN,PG Label,PG Acronym,PG Description,EDP,DP,PF,PS,Multipacket,Transmission Rate,...,Scale Factor\n(value only),Offset\n(value only),Range Maximum\n(value only),Length Minimum\n(bits),Length Maximum\n(bits),SP Document,PG Document,SP Created or Modified Date,PG Created or Modified Date,SP to PG Mapping Created or Modified Date
0,0,Torque/Speed Control 1,TSC1,NOTE - Retarder may be disabled by commanding ...,0,0,0,DA,No,To engine: Control Purpose dependent or 10 ms_...,...,1.000,0.0,3.000,2,2,J1939DA,J1939DA,2017-08-10,2017-08-10,1998-10-01
1,0,Torque/Speed Control 1,TSC1,NOTE - Retarder may be disabled by commanding ...,0,0,0,DA,No,To engine: Control Purpose dependent or 10 ms_...,...,1.000,0.0,3.000,2,2,J1939DA,J1939DA,2003-11-30,2017-08-10,1998-10-01
2,0,Torque/Speed Control 1,TSC1,NOTE - Retarder may be disabled by commanding ...,0,0,0,DA,No,To engine: Control Purpose dependent or 10 ms_...,...,1.000,0.0,3.000,2,2,J1939DA,J1939DA,NaT,2017-08-10,1998-10-01
3,0,Torque/Speed Control 1,TSC1,NOTE - Retarder may be disabled by commanding ...,0,0,0,DA,No,To engine: Control Purpose dependent or 10 ms_...,...,0.125,0.0,8031.875,16,16,J1939DA,J1939DA,2003-11-30,2017-08-10,1998-10-01
4,0,Torque/Speed Control 1,TSC1,NOTE - Retarder may be disabled by commanding ...,0,0,0,DA,No,To engine: Control Purpose dependent or 10 ms_...,...,1.000,-125.0,125.000,8,8,J1939DA,J1939DA,2003-11-30,2017-08-10,1998-10-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,J1939-73,NaN,2002-04-14,NaT,NaT
30050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,J1939-73,NaN,2002-04-14,NaT,NaT
30051,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,J1939-73,NaN,2002-04-14,NaT,NaT
30052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,J1939-73,NaN,2002-04-14,NaT,NaT


### Parameter Group Numbers

In [9]:
pgns = spn_df.groupby(by='PGN')
pgns.groups

{0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 256: [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34], 512: [35], 768: [36], 1024: [37, 38, 39, 40, 41, 42, 43, 44, 45], 1280: [46], 1536: [47], 1792: [48, 49, 50, 51], 2048: [52, 53, 54, 55], 2304: [56, 57], 2560: [58, 59, 60, 61, 62, 63, 64, 65], 2816: [66, 67, 68], 3072: [69, 70, 71], 3328: [72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88], 3584: [89, 90, 91, 92, 93, 94, 95, 96, 97], 3840: [98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109], 4096: [110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121], 4352: [122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133], 4608: [134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145], 4864: [146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157], 5120: [158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169], 5376: [170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181], 5632: [182, 183, 184, 185, 18

In [10]:
reduced_pgns = (spn_df.drop_duplicates('PGN')[['PGN','PG Label','Transmission Rate','PG Acronym','PG Data Length']])
reduced_pgns.dropna(how='any',subset=["PGN"],inplace=True)
reduced_pgns.set_index('PGN',inplace=True)
reduced_pgns.rename(columns={'PG Label': 'Name', 
                             'PG Acronym': 'Label',
                             'PG Data Length':'PGNLength',
                             'Transmission Rate':"Rate"},
                    inplace=True)
reduced_pgns

,Name,Rate,Label,PGNLength
PGN,,,,
0,Torque/Speed Control 1,To engine: Control Purpose dependent or 10 ms_...,TSC1,8
256,Transmission Control 1,System dependent; either 50 ms as needed for a...,TC1,8
512,Electronic Brake System #1/1,NaN,EBS11,NaN
768,Electronic Brake System #2/1,NaN,EBS21,NaN
1024,External Brake Request,When active: 20 ms; else 200 ms,XBR,8
...,...,...,...,...
131067,Proprietary B - Page 1,NaN,PropB1_FB,NaN
131068,Proprietary B - Page 1,NaN,PropB1_FC,NaN
131069,Proprietary B - Page 1,NaN,PropB1_FD,NaN


In [11]:
pgn_dict = reduced_pgns.to_dict(orient='index')
for pgn_key in reduced_pgns.index:
    pgn = int(pgn_key)
    pgn_dict[pgn]["SPNs"] = sorted(spn_df["SPN"][pgns.groups[pgn]].dropna().to_list())
pgn_dict

{0: {'Name': 'Torque/Speed Control 1',
  'Rate': 'To engine: Control Purpose dependent or 10 ms_x000D_\nTo retarder: 50 ms',
  'Label': 'TSC1',
  'PGNLength': 8,
  'SPNs': [518, 695, 696, 897, 898, 3349, 3350, 4191, 4206, 4207]},
 256: {'Name': 'Transmission Control 1',
  'Rate': 'System dependent; either 50 ms as needed for active control, or as a continuous 50 ms periodic broadcast.',
  'Label': 'TC1',
  'PGNLength': 8,
  'SPNs': [525,
   681,
   682,
   683,
   684,
   685,
   686,
   687,
   688,
   689,
   690,
   691,
   1852,
   1853,
   1854,
   1855,
   2985,
   4242,
   4246,
   4247,
   4248,
   4249,
   4255,
   5762,
   7695]},
 512: {'Name': 'Electronic Brake System #1/1',
  'Rate': nan,
  'Label': 'EBS11',
  'PGNLength': nan,
  'SPNs': []},
 768: {'Name': 'Electronic Brake System #2/1',
  'Rate': nan,
  'Label': 'EBS21',
  'PGNLength': nan,
  'SPNs': []},
 1024: {'Name': 'External Brake Request',
  'Rate': 'When active: 20 ms; else 200 ms',
  'Label': 'XBR',
  'PGNLength

In [12]:
pgn_dict[65265]

{'Name': 'Cruise Control/Vehicle Speed 1',
 'Rate': '100 ms',
 'Label': 'CCVS1',
 'PGNLength': 8,
 'SPNs': [69,
  70,
  84,
  86,
  527,
  595,
  596,
  597,
  598,
  599,
  600,
  601,
  602,
  966,
  967,
  968,
  976,
  1237,
  1633,
  3807]}

In [13]:
pgn_dict[0]

{'Name': 'Torque/Speed Control 1',
 'Rate': 'To engine: Control Purpose dependent or 10 ms_x000D_\nTo retarder: 50 ms',
 'Label': 'TSC1',
 'PGNLength': 8,
 'SPNs': [518, 695, 696, 897, 898, 3349, 3350, 4191, 4206, 4207]}

In [14]:
try:
    pgn_dict[1]
except KeyError:
    print("Key Error - Entry does not exist.")

Key Error - Entry does not exist.


## Suspect Parameter Numbers

In [15]:
#%pip install --upgrade --user pandas

In [16]:
reduced_spns = spn_df.drop_duplicates('SPN')[['SPN',
                                              'PGN',
                                              'SP Label',
                                              'Transmission Rate',
                                              'PG Acronym',
                                              'SP Length',
                                              'SLOT Identifier',
                                              'Data Range'
                                             ]]
reduced_spns.dropna(how='any',subset=["SPN"],inplace=True)
reduced_spns.set_index('SPN',inplace=True)
reduced_spns.rename(columns={'SP Label': 'Name', 
                             'PG Acronym': 'Acronym',
                             'PG Data Length':'PGNLength',
                             'SP Length':'SPNLength',
                             'Transmission Rate':"TransmissionRate"},
                    inplace=True)
reduced_spns

,PGN,Name,TransmissionRate,Acronym,SPNLength,SLOT Identifier,Data Range
SPN,,,,,,,
695,0,Engine Override Control Mode,To engine: Control Purpose dependent or 10 ms_...,TSC1,2 bits,87,0 to 3
696,0,Engine Requested Speed Control Conditions,To engine: Control Purpose dependent or 10 ms_...,TSC1,2 bits,87,0 to 3
897,0,Override Control Mode Priority,To engine: Control Purpose dependent or 10 ms_...,TSC1,2 bits,87,0 to 3
898,0,Engine Requested Speed/Speed Limit,To engine: Control Purpose dependent or 10 ms_...,TSC1,2 bytes,76,0 to 8 031.875 rpm
518,0,Engine Requested Torque/Torque Limit,To engine: Control Purpose dependent or 10 ms_...,TSC1,1 byte,45,-125 to 125 %
...,...,...,...,...,...,...,...
524283,NaN,Manufacturer Assignable SPN,NaN,NaN,NaN,NaN,NaN
524284,NaN,Manufacturer Assignable SPN,NaN,NaN,NaN,NaN,NaN
524285,NaN,Manufacturer Assignable SPN,NaN,NaN,NaN,NaN,NaN


In [17]:
spn_dict = reduced_spns.to_dict(orient='index')
spn_dict

{695: {'PGN': 0,
  'Name': 'Engine Override Control Mode',
  'TransmissionRate': 'To engine: Control Purpose dependent or 10 ms_x000D_\nTo retarder: 50 ms',
  'Acronym': 'TSC1',
  'SPNLength': '2 bits',
  'SLOT Identifier': 87,
  'Data Range': '0 to 3'},
 696: {'PGN': 0,
  'Name': 'Engine Requested Speed Control Conditions',
  'TransmissionRate': 'To engine: Control Purpose dependent or 10 ms_x000D_\nTo retarder: 50 ms',
  'Acronym': 'TSC1',
  'SPNLength': '2 bits',
  'SLOT Identifier': 87,
  'Data Range': '0 to 3'},
 897: {'PGN': 0,
  'Name': 'Override Control Mode Priority',
  'TransmissionRate': 'To engine: Control Purpose dependent or 10 ms_x000D_\nTo retarder: 50 ms',
  'Acronym': 'TSC1',
  'SPNLength': '2 bits',
  'SLOT Identifier': 87,
  'Data Range': '0 to 3'},
 898: {'PGN': 0,
  'Name': 'Engine Requested Speed/Speed Limit',
  'TransmissionRate': 'To engine: Control Purpose dependent or 10 ms_x000D_\nTo retarder: 50 ms',
  'Acronym': 'TSC1',
  'SPNLength': '2 bytes',
  'SLOT Id

In [18]:
for k,v in spn_dict.items():
    slot = v["SLOT Identifier"]
    try:
        spn_dict[k].update(slot_dict[slot])
    except KeyError:
        pass
spn_dict

{695: {'PGN': 0,
  'Name': 'Engine Override Control Mode',
  'TransmissionRate': 'To engine: Control Purpose dependent or 10 ms_x000D_\nTo retarder: 50 ms',
  'Acronym': 'TSC1',
  'SPNLength': '2 bits',
  'SLOT Identifier': 87,
  'Data Range': '0 to 3',
  'byte length': '2 bits',
  'Unit': nan,
  'Resolution': 1.0,
  'Offset': 0.0,
  'OperationalHigh': 3,
  'Length': 2.0,
  'Max Bits': 2.0},
 696: {'PGN': 0,
  'Name': 'Engine Requested Speed Control Conditions',
  'TransmissionRate': 'To engine: Control Purpose dependent or 10 ms_x000D_\nTo retarder: 50 ms',
  'Acronym': 'TSC1',
  'SPNLength': '2 bits',
  'SLOT Identifier': 87,
  'Data Range': '0 to 3',
  'byte length': '2 bits',
  'Unit': nan,
  'Resolution': 1.0,
  'Offset': 0.0,
  'OperationalHigh': 3,
  'Length': 2.0,
  'Max Bits': 2.0},
 897: {'PGN': 0,
  'Name': 'Override Control Mode Priority',
  'TransmissionRate': 'To engine: Control Purpose dependent or 10 ms_x000D_\nTo retarder: 50 ms',
  'Acronym': 'TSC1',
  'SPNLength': '2

In [19]:
spn_dict[190]

{'PGN': 61444,
 'Name': 'Engine Speed',
 'TransmissionRate': 'Fixed rate of 10 to 50 ms or engine speed dependent',
 'Acronym': 'EEC1',
 'SPNLength': '2 bytes',
 'SLOT Identifier': 76,
 'Data Range': '0 to 8 031.875 rpm',
 'byte length': '2 bytes',
 'Unit': 'rpm',
 'Resolution': 0.125,
 'Offset': 0.0,
 'OperationalHigh': 8031.875,
 'Length': 16.0,
 'Max Bits': 16.0}

In [20]:
spn_dict[84]

{'PGN': 65265,
 'Name': 'Wheel-Based Vehicle Speed',
 'TransmissionRate': '100 ms',
 'Acronym': 'CCVS1',
 'SPNLength': '2 bytes',
 'SLOT Identifier': 71,
 'Data Range': '0 to 250.996 093 75 km/h',
 'byte length': '2 bytes',
 'Unit': 'km/h',
 'Resolution': 0.00390625,
 'Offset': 0.0,
 'OperationalHigh': 250.99609375,
 'Length': 16.0,
 'Max Bits': 16.0}

In [21]:
spn_dict[237]

{'PGN': 65260,
 'Name': 'Vehicle Identification Number',
 'TransmissionRate': 'On request',
 'Acronym': 'VI',
 'SPNLength': 'Variable - 0 to 200 bytes followed by "*" delimiter',
 'SLOT Identifier': 108,
 'Data Range': '0 to 255 per byte',
 'byte length': 'Variable - 0 to 200 bytes followed by "*" delimiter',
 'Unit': nan,
 'Resolution': 1.0,
 'Offset': 0.0,
 'OperationalHigh': 255,
 'Length': 0.0,
 'Max Bits': 1600.0}

### US Units

In [31]:
us_slot_dict = slot_dict.copy()
for k,v in slot_dict.items():
    if v["Unit"] == 'km':
        factor = 0.6213712
        us_slot_dict[k]["Unit"] = 'miles'
        us_slot_dict[k]["Resolution"] = v["Resolution"]*factor
        us_slot_dict[k]["Offset"] = v["Offset"]*factor
        us_slot_dict[k]["OperationalHigh"] = v["OperationalHigh"]*factor
    elif v["Unit"] == 'km/h':
        factor = 0.6213712
        us_slot_dict[k]["Unit"] = 'mph'
        us_slot_dict[k]["Resolution"] = v["Resolution"]*factor
        us_slot_dict[k]["Offset"] = v["Offset"]*factor
        us_slot_dict[k]["OperationalHigh"] = v["OperationalHigh"]*factor
    elif v["Unit"] == 'kpa':
        factor = 0.1450377
        us_slot_dict[k]["Unit"] = 'psi'
        us_slot_dict[k]["Resolution"] = v["Resolution"]*factor
        us_slot_dict[k]["Offset"] = v["Offset"]*factor
        us_slot_dict[k]["OperationalHigh"] = v["OperationalHigh"]*factor
    elif v["Unit"] == 'm':
        factor = 3.28084
        us_slot_dict[k]["Unit"] = 'ft'
        us_slot_dict[k]["Resolution"] = v["Resolution"]*factor
        us_slot_dict[k]["Offset"] = v["Offset"]*factor
        us_slot_dict[k]["OperationalHigh"] = v["OperationalHigh"]*factor
us_slot_dict[74]

{'byte length': '1 byte',
 'Unit': 'mph',
 'Resolution': 0.03794375,
 'Offset': -4.74296875,
 'OperationalHigh': 4.74296875,
 'Length': 8.0,
 'Max Bits': 8.0}

In [33]:
#US Version
for k,v in spn_dict.items():
    slot = v["SLOT Identifier"]
    try:
        spn_dict[k].update(us_slot_dict[slot])
    except KeyError:
        pass
spn_dict[84]

{'PGN': 65265,
 'Name': 'Wheel-Based Vehicle Speed',
 'TransmissionRate': '100 ms',
 'Acronym': 'CCVS1',
 'SPNLength': '2 bytes',
 'SLOT Identifier': 71,
 'Data Range': '0 to 250.996 093 75 km/h',
 'byte length': '2 bytes',
 'Unit': 'mph',
 'Resolution': 0.002371484375,
 'Offset': 0.0,
 'OperationalHigh': 152.379728515625,
 'Length': 16.0,
 'Max Bits': 16.0}

## Source Addresses

In [23]:
#Create a pandas data frame with the Source Addresses
with open('J1939DA OCT2021.xlsx', 'rb') as io:
    global_sa_df = pandas.read_excel(io, sheet_name='Global Source Addresses (B2)',
                                 usecols="B:D", 
                                 index_col=0,
                                 header = 3)
global_sa_df

,Name,Notes
Source Address ID,,
0,Engine #1,The #1 on the Engine CA is to identify that th...
1,Engine #2,The #2 on the Engine CA is to identify that th...
2,Turbocharger,Turbocharger used on the engine. May only be ...
3,Transmission #1,The first transmission - may only be used for ...
4,Transmission #2,The second PA available for use for the functi...
...,...,...
251,On-Board Data Logger,NaN
252,Reserved for Experimental Use,NaN
253,Reserved for OEM,NaN


In [24]:
#Create a pandas data frame with the Source Addresses
with open('J1939DA OCT2021.xlsx', 'rb') as io:
    ig1_sa_df = pandas.read_excel(io, sheet_name='IG1 Source Addresses (B3)',
                                 usecols="B:D", 
                                 index_col=0,
                                 header = 3)
ig1_sa_df

,Name,Notes
Source Address ID,,
128,thru 155 are reserved for future assignment by...,Used for dynamic address assignment
156,Automated Driving Controller 2,Second Automated Driving Controller _x000D_\n_...
157,Electric Propulsion Control Unit #3,Third on-board device converting torque comman...
158,Automated Driving Controller 1,Automated Driving Controller_x000D_\n_x000D_\n...
159,Roadway Information System,Controller applications that use this source a...
...,...,...
243,Battery Pack Monitor #1,Device to monitor battery pack #1
244,Battery Pack Monitor #2 / APU #4,SA for either the device to monitor battery pa...
245,Battery Pack Monitor #3 / APU #3,SA for either the device to monitor battery pa...


In [25]:
sa_df = global_sa_df.merge(ig1_sa_df,how='outer',on=["Source Address ID","Name","Notes"], sort=True)
sa_df

,Name,Notes
Source Address ID,,
0,Engine #1,The #1 on the Engine CA is to identify that th...
1,Engine #2,The #2 on the Engine CA is to identify that th...
2,Turbocharger,Turbocharger used on the engine. May only be ...
3,Transmission #1,The first transmission - may only be used for ...
4,Transmission #2,The second PA available for use for the functi...
...,...,...
251,On-Board Data Logger,NaN
252,Reserved for Experimental Use,NaN
253,Reserved for OEM,NaN
